<a href="https://colab.research.google.com/github/Udhayabas/RAG/blob/main/huggingface_with_FAISS_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers langchain faiss-cpu


In [3]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [2]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Initialize HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-5-612665a3bcc7>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from google.colab import files

# Upload PDF files
uploaded = files.upload()


Saving BELT STOCK ECHS.pdf to BELT STOCK ECHS (2).pdf


In [7]:
from langchain_community.document_loaders import PyPDFLoader

# Create a list to store documents from all PDF files
all_docs = []

# If you want to process specific files (not uploaded), simply provide the correct file path
# Use PyPDFLoader to load the file content
loader = PyPDFLoader('/content/BELT STOCK ECHS.pdf')  # Path to the specific PDF file

# Load and extend the documents from the file
docs = loader.load()
all_docs.extend(docs)  # Add the loaded documents to the list

# Now, all_docs contains the content of the specified PDF file


In [6]:
!pip install pypdf

In [9]:
# Create a FAISS vector store using the Hugging Face embeddings
db = FAISS.from_documents(docs, embedding_model)


In [10]:
db

In [11]:
# Check how many documents are in the FAISS vector store
print(f"Number of documents in the vector store: {db.index.ntotal}")

Number of documents in the vector store: 1


In [12]:
# Define your query
query = "tell how much metrs of belt in stock st2000,width2000mm"

# Embed the query using the same embedding model
# Instead of embedding the query directly, pass it as a single-element list to `embed_query`
# query_embedding = embedding_model.embed_query(query)  <- This was the problem
# The HuggingFaceEmbeddings class's embed_query method expects a string, not a list.
query_embedding = embedding_model.embed_query(query)

# Perform similarity search in the FAISS vector store
similar_docs = db.similarity_search(query, k=1)  # k defines the number of top results to return

# Display the results
for i, doc in enumerate(similar_docs):
    print(f"Result {i+1}: {doc.page_content}")

Result 1: S.NO BELT 
STRENGTH 
AND BELT 
WIDTH  STOCK BELT 
(METRS) BELT DRUM 
DETAILS BELT MAKE REMARKS 
250Mtrs-06no continental 
300mtrs-03no FORECH 
190Mtrs-01 continental 
200Mtrs-03 FORECH 
250mtrs-01 FORECH 
83MTRS-1 FORECH 
250MTRS-6 continental-2 
FORECH-4 
200MTRS-1 FORECH 
76MTRS-1 FORECH 
300mtrs-03no FORECH 
200 MTRS-1 continental 
8 EP800/4 
(1600mm) 210 210 Mtrs-01no continental 90mtrs given to 
stage2 
250mtrs-01no 
10 ST 1400 
(1600 mm) 90 90mtrs-01no phonix ETPS BELT FORECH 
FORECH ECHS  BELT STOCK DETAILS               30/09/2024 
250metrs sent 
 to stage2 300mtrs-04no 
200mtrs-01no FORECH 
continental 
continental 
FORECH 
300mtrs-02no 
250mtrs-01no FORECH 
FORECH 
9 ST 1150 
(1600 mm) 250 PO BELT 
 RECEIVED on  
10/09/24 
850 MTRS 
FORECH 
200MTRS-1 
146 MTRS-1 250mtrs-01no 
187 mtrs-01no 63 metrs sfd1A 
REPLACED 
27/09/24 
27 mtrs imc-1, 
(two time) 
18/09/24 1 ST 2000 
( 2000 mm) 3523 
4 EP 800/4 
(2400mm) 437 3 ST 1400 
( 2000mm) 500 
TOTAL BELT STOCK IN HAND= 9

# fine tuning model below **steps**

In [13]:
from transformers import AutoModel, AutoTokenizer

# Load the pre-trained model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# You would need a dataset and training script to fine-tune the model here


In [25]:
from langchain_core.documents import Document

# Add metadata to your documents and create Document objects
docs_with_metadata = [
    Document(page_content="ST2000 belt, 2000mm width, 3523 meters in stock.", metadata={"type": "stock", "category": "belt"}),
    Document(page_content="ST1600 belt, 2000mm width, 1776 meters in stock.", metadata={"type": "stock", "category": "belt"})
]

# Add documents with metadata to the FAISS store
db.add_documents(docs_with_metadata)

# Now you can search by query and filter by metadata
similar_docs = db.similarity_search(query, k=5, filter={"category": "belt"})

In [24]:
similar_docs = db.similarity_search(query, k=5)  # Get top 5 results


In [8]:
# Use a different embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-v4")


In [19]:
# Question about belt stock
question = "How many meters of EP800/4,2400MMWIDTH belts are in stock?"

# Perform search
results = db.similarity_search(question, k=2)

# Display the top 2 results
for i, doc in enumerate(results):
    print(f"Result {i+1}:\n{doc.page_content}\n")


Result 1:
S.NO BELT 
STRENGTH 
AND BELT 
WIDTH  STOCK BELT 
(METRS) BELT DRUM 
DETAILS BELT MAKE REMARKS 
250Mtrs-06no continental 
300mtrs-03no FORECH 
190Mtrs-01 continental 
200Mtrs-03 FORECH 
250mtrs-01 FORECH 
83MTRS-1 FORECH 
250MTRS-6 continental-2 
FORECH-4 
200MTRS-1 FORECH 
76MTRS-1 FORECH 
300mtrs-03no FORECH 
200 MTRS-1 continental 
8 EP800/4 
(1600mm) 210 210 Mtrs-01no continental 90mtrs given to 
stage2 
250mtrs-01no 
10 ST 1400 
(1600 mm) 90 90mtrs-01no phonix ETPS BELT FORECH 
FORECH ECHS  BELT STOCK DETAILS               30/09/2024 
250metrs sent 
 to stage2 300mtrs-04no 
200mtrs-01no FORECH 
continental 
continental 
FORECH 
300mtrs-02no 
250mtrs-01no FORECH 
FORECH 
9 ST 1150 
(1600 mm) 250 PO BELT 
 RECEIVED on  
10/09/24 
850 MTRS 
FORECH 
200MTRS-1 
146 MTRS-1 250mtrs-01no 
187 mtrs-01no 63 metrs sfd1A 
REPLACED 
27/09/24 
27 mtrs imc-1, 
(two time) 
18/09/24 1 ST 2000 
( 2000 mm) 3523 
4 EP 800/4 
(2400mm) 437 3 ST 1400 
( 2000mm) 500 
TOTAL BELT STOCK IN HAND= 9

In [36]:
# Check the documents stored in the FAISS index
for i, doc in enumerate(docs_with_metadata):
    print(f"Document {i+1}: {doc.page_content}")


Document 1: ST2000 belt, 2000mm width, 3523 meters in stock.
Document 2: ST1600 belt, 2000mm width, 1776 meters in stock.


In [15]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document

# Load the PDF content
loader = PyPDFLoader('/content/BELT STOCK ECHS.pdf')
pdf_documents = loader.load()

# Inspect the loaded documents
for i, doc in enumerate(pdf_documents):
    print(f"Document {i+1}: {doc.page_content}")


Document 1: S.NO BELT 
STRENGTH 
AND BELT 
WIDTH  STOCK BELT 
(METRS) BELT DRUM 
DETAILS BELT MAKE REMARKS 
250Mtrs-06no continental 
300mtrs-03no FORECH 
190Mtrs-01 continental 
200Mtrs-03 FORECH 
250mtrs-01 FORECH 
83MTRS-1 FORECH 
250MTRS-6 continental-2 
FORECH-4 
200MTRS-1 FORECH 
76MTRS-1 FORECH 
300mtrs-03no FORECH 
200 MTRS-1 continental 
8 EP800/4 
(1600mm) 210 210 Mtrs-01no continental 90mtrs given to 
stage2 
250mtrs-01no 
10 ST 1400 
(1600 mm) 90 90mtrs-01no phonix ETPS BELT FORECH 
FORECH ECHS  BELT STOCK DETAILS               30/09/2024 
250metrs sent 
 to stage2 300mtrs-04no 
200mtrs-01no FORECH 
continental 
continental 
FORECH 
300mtrs-02no 
250mtrs-01no FORECH 
FORECH 
9 ST 1150 
(1600 mm) 250 PO BELT 
 RECEIVED on  
10/09/24 
850 MTRS 
FORECH 
200MTRS-1 
146 MTRS-1 250mtrs-01no 
187 mtrs-01no 63 metrs sfd1A 
REPLACED 
27/09/24 
27 mtrs imc-1, 
(two time) 
18/09/24 1 ST 2000 
( 2000 mm) 3523 
4 EP 800/4 
(2400mm) 437 3 ST 1400 
( 2000mm) 500 
TOTAL BELT STOCK IN HAND=

In [16]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store from the parsed PDF documents
db = FAISS.from_documents(pdf_documents, embedding_model)


In [17]:
# Define your question (prompt)
question = "How many meters of 2400mm EP800/4 belts are in stock as per the PDF?"

# Perform a similarity search
results = db.similarity_search(question, k=1)

# Display the top results
for i, doc in enumerate(results):
    print(f"Result {i+1}: {doc.page_content}")


Result 1: S.NO BELT 
STRENGTH 
AND BELT 
WIDTH  STOCK BELT 
(METRS) BELT DRUM 
DETAILS BELT MAKE REMARKS 
250Mtrs-06no continental 
300mtrs-03no FORECH 
190Mtrs-01 continental 
200Mtrs-03 FORECH 
250mtrs-01 FORECH 
83MTRS-1 FORECH 
250MTRS-6 continental-2 
FORECH-4 
200MTRS-1 FORECH 
76MTRS-1 FORECH 
300mtrs-03no FORECH 
200 MTRS-1 continental 
8 EP800/4 
(1600mm) 210 210 Mtrs-01no continental 90mtrs given to 
stage2 
250mtrs-01no 
10 ST 1400 
(1600 mm) 90 90mtrs-01no phonix ETPS BELT FORECH 
FORECH ECHS  BELT STOCK DETAILS               30/09/2024 
250metrs sent 
 to stage2 300mtrs-04no 
200mtrs-01no FORECH 
continental 
continental 
FORECH 
300mtrs-02no 
250mtrs-01no FORECH 
FORECH 
9 ST 1150 
(1600 mm) 250 PO BELT 
 RECEIVED on  
10/09/24 
850 MTRS 
FORECH 
200MTRS-1 
146 MTRS-1 250mtrs-01no 
187 mtrs-01no 63 metrs sfd1A 
REPLACED 
27/09/24 
27 mtrs imc-1, 
(two time) 
18/09/24 1 ST 2000 
( 2000 mm) 3523 
4 EP 800/4 
(2400mm) 437 3 ST 1400 
( 2000mm) 500 
TOTAL BELT STOCK IN HAND= 9

In [42]:
# Add metadata to parsed PDF documents (manually or programmatically)
docs_with_metadata = [
    Document(page_content=doc.page_content, metadata={"category": "stock", "product": "EP800/4"})
    for doc in pdf_documents
]

# Add documents with metadata to FAISS
db.add_documents(docs_with_metadata)

# Now you can query with metadata filtering
results = db.similarity_search(question, k=2, filter={"product": "EP800/4"})


In [18]:
# Get more results (e.g., top 5 results)
results = db.similarity_search(question, k=2)

# Display the results
for i, doc in enumerate(results):
    print(f"Result {i+1}: {doc.page_content}")


Result 1: S.NO BELT 
STRENGTH 
AND BELT 
WIDTH  STOCK BELT 
(METRS) BELT DRUM 
DETAILS BELT MAKE REMARKS 
250Mtrs-06no continental 
300mtrs-03no FORECH 
190Mtrs-01 continental 
200Mtrs-03 FORECH 
250mtrs-01 FORECH 
83MTRS-1 FORECH 
250MTRS-6 continental-2 
FORECH-4 
200MTRS-1 FORECH 
76MTRS-1 FORECH 
300mtrs-03no FORECH 
200 MTRS-1 continental 
8 EP800/4 
(1600mm) 210 210 Mtrs-01no continental 90mtrs given to 
stage2 
250mtrs-01no 
10 ST 1400 
(1600 mm) 90 90mtrs-01no phonix ETPS BELT FORECH 
FORECH ECHS  BELT STOCK DETAILS               30/09/2024 
250metrs sent 
 to stage2 300mtrs-04no 
200mtrs-01no FORECH 
continental 
continental 
FORECH 
300mtrs-02no 
250mtrs-01no FORECH 
FORECH 
9 ST 1150 
(1600 mm) 250 PO BELT 
 RECEIVED on  
10/09/24 
850 MTRS 
FORECH 
200MTRS-1 
146 MTRS-1 250mtrs-01no 
187 mtrs-01no 63 metrs sfd1A 
REPLACED 
27/09/24 
27 mtrs imc-1, 
(two time) 
18/09/24 1 ST 2000 
( 2000 mm) 3523 
4 EP 800/4 
(2400mm) 437 3 ST 1400 
( 2000mm) 500 
TOTAL BELT STOCK IN HAND= 9